In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import preprocessing

sys.path.append('./src')


import myclassify
reload(myclassify)
from myclassify import MyFeatureSet
from myclassify import MyCountTable
from myclassify import merge_two_cat_columns
from myclassify import np_combine_rare

%matplotlib inline 
# to make reproducible results
SEED = 1234

In [ ]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

ytrain = df_train['response'].values
idtest = df_test['id'].values

In [ ]:
reload(myclassify)

orig_fset = myclassify.df_to_fset([df_train, df_test], drop_list = ['id'])

In [ ]:
reload(myclassify)


orig_fset.show_basic_statics()

In [ ]:
# try running rf on orig_fset

reload(myclassify)



myrf_params = {'n_estimators':100,'max_features':0.4,'max_depth':11,'min_samples_split':3,
                     'n_jobs':-1,'random_state':SEED,'criterion':'entropy', 'class_weight':'balanced_subsample'}

myrf = myclassify.MyRandomForest(myrf_params)

# myxgb = myclassify.MyXGBoost(myxgb_params)

myrf_cv_score = myclassify.strat_cv_score(myrf, orig_fset.Xtrain, ytrain)

print np.mean(myrf_cv_score), np.std(myrf_cv_score)

myrf.fit(orig_fset.Xtrain, ytrain)

myrf.list_feature_importance(orig_fset.fname_list)

myrf.plt_feature_importance(orig_fset.fname_list)


In [ ]:
# use xgb on orig_fset

myxgb_params = {'objective': 'binary:logistic', 'subsample': .9, 'nthread': 4, 'seed': SEED, 'num_round':500,
                   'learning_rate': 0.02, 'n_estimators': 1000, 'colsample_bylevel':0.7, 
                   'max_depth': 6,'gamma': 0.6, 'colsample_bytree':0.7, 'min_child_weight':0.,
                      'lambda': 1.0, 'alpha': 0, 'eval_metric':'logloss'}

myxgb = myclassify.MyXGBoost(myxgb_params)

myxgb_cv_score = myclassify.strat_cv_score(myxgb, orig_fset.Xtrain, ytrain)

print np.mean(myxgb_cv_score), np.std(myxgb_cv_score)

myxgb.fit(orig_fset.Xtrain, ytrain)

myxgb.list_feature_importance(orig_fset.fname_list)

myxgb.plt_feature_importance(orig_fset.fname_list)

In [ ]:
# merge var20, var11, var17, var9, var10 and do label encoding on var12 and combine rare
reload(myclassify)
num_list = ['var18', 'var12']
merge_list = ['le_var9', 'var10', 'le_var11', 'le_var17', 'le_var20']

integer_list = [col for col in orig_fset.feature_names() \
                    if issubclass(orig_fset.ftype_list[orig_fset.fname_list.index(col)], np.integer)]
relabel_list = [col for col in orig_fset.feature_names() if col not in num_list \
                      and not issubclass(orig_fset.ftype_list[orig_fset.fname_list.index(col)], np.integer)]

%time cat_int_fset = orig_fset[integer_list]

%time cat_float_fset = orig_fset[relabel_list].label_encode(relabel_list)

%time num_fset = orig_fset[num_list]

merge_fset = cat_int_fset + cat_float_fset

merge_fset['merge_var9_10_11_17_20'] = merge_fset.merge_multiple_cat_columns(merge_list)

merge_fset = merge_fset.drop(merge_list)
%time merge_fset = merge_fset + orig_fset['var18'] + orig_fset['var12'].label_encode(['var12']).combine_rare()

print merge_fset.fname_list

In [ ]:
myxgb_params = {'objective': 'binary:logistic', 'subsample': .9, 'nthread': 4, 'seed': SEED, 'num_round':500,
                   'learning_rate': 0.02, 'n_estimators': 1000, 'colsample_bylevel':0.7, 
                   'max_depth': 6,'gamma': 0.6, 'colsample_bytree':0.7, 'min_child_weight':0.,
                      'lambda': 1.0, 'alpha': 0, 'eval_metric':'logloss'}

myxgb = myclassify.MyXGBoost(myxgb_params)

myxgb_cv_score = myclassify.strat_cv_score(myxgb, merge_fset.Xtrain, ytrain)

print np.mean(myxgb_cv_score), np.std(myxgb_cv_score)

myxgb.fit(merge_fset.Xtrain, ytrain)

myxgb.list_feature_importance(merge_fset.fname_list)

myxgb.plt_feature_importance(merge_fset.fname_list)

In [ ]:
myxgb_ypred = myxgb.predict_proba(merge_fset.Xtest)

submission = pd.DataFrame({"id":idtest, "response":myxgb_ypred})
submission = submission[['id', 'response']]
submission.to_csv("xgb_submission.csv", index=False)